# Insula API Data Collections

## Overview

This document serves as a quick reference guide of how to access Insula's APIs, using Python. This guide will show how to create a collection and upload data to it.


***Steps***:
1. Preparation
2. Create a Collection
3. Upload Data to a Collection

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each http request.
Be sure to put a valid `USERNAME/TOKEN` pair, and check that the domain inside `BASE_URL` is still valid.

In [ ]:
pip install InsulaWorkflowClient

In [ ]:
import requests
import base64
import json
import getpass

In [ ]:
USERNAME=getpass.getpass()

In [ ]:
PASSWORD=getpass.getpass()

In [ ]:
from InsulaWorkflowClient import InsulaOpenIDConnect

BASE_URL="https://insula.earth"

insulaAuth: InsulaOpenIDConnect = InsulaOpenIDConnect(
        authorization_endpoint="https://identity.insula.earth/realms/eopaas/protocol/openid-connect/auth",
        token_endpoint="https://identity.insula.earth/realms/eopaas/protocol/openid-connect/token",
        redirect_uri="http://localhost:9207/auth",
        client_id="api-client"
    )
insulaAuth.set_user_credentials(username=USERNAME, password=PASSWORD)

bearer = insulaAuth.get_authorization_header()
print(bearer)
HEADERS={'Authorization': bearer }

## Create a Collection

For creating a new collection we need to pass a JSON object with its details along with the POST HTTP request.
Inside `coll` you can see how the JSON object is made, feel free to modify its values.

> Be mindful when running this one as if a collection with the same name is already present in the platform the command will fail with HTTP 409, so try with a different collection or delete the old one to test it a second time.

In [ ]:
collection_name='test-jupytr-coll'
coll={'name': collection_name, 'description': "no", 'fileType': "REFERENCE_DATA", 'productsType': "no"}

url=BASE_URL + "/secure/api/v2.0/collections/"
run_request=requests.post(url, headers=HEADERS, json=coll, verify=True)
run_request_dict = json.loads(run_request.text)

if run_request.status_code>=400: print(str(run_request.status_code)+" ERROR: Could not create Collection")
else: print(str(run_request.status_code) +" Collection successfully created")

In [ ]:
run_request_dict

Now that the collection exists we can search for it with the call below, we will need it's identifier to upload a file to it.

In [ ]:
url = BASE_URL+'/secure/api/v2.0/collections/search/parametricFind?filter='+collection_name
run_request=requests.get(url, headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

In [ ]:
collection_identifier=run_request_dict['_embedded']['collections'][0]['identifier']
collection_identifier

## Upload Data to a Collection
For this step a file (`filename`) is created form scratch and uploaded, feel free to modify it with an already existing one and if you do remove the `truncate()` line.

The object (`uploads`) has to be created with the structure below and passed under the `files` field of the http request to work correctly.

We're using the `refData` API to upload also specifying the collection and the file type as the collection we created before was a reference data collection.

In [ ]:
file_type='OTHER'

filename="uploadTestFile.tif"
metadata={} #put optional metadata fields here

with open(filename, "w+b") as upload:
    upload.truncate(1024) #just to produce a 1kb file
    uploads={
        'file':(filename, upload, 'image/tif'),
        'userProperties': (None, json.dumps(metadata), 'application/json')
    }
    
    url=BASE_URL + f"/secure/api/v2.0/platformFiles/refData?fileType={file_type}&collection={collection_identifier}"
    
    run_request=requests.post(url, headers=HEADERS, files=uploads)
    run_request.raise_for_status()
    run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict